# Firewalls

### General
- Principle of Least Priveledge
- Defense in depth
- Keep it simple
- Log everything
- Test

**Traffic Types**
- Ingress
    - Inbound traffic
- Egress
    - Outbound traffic
    - Use isolation to protect from sources within a network
    - Example: wannacry worm must travel through a network interface

### iptables
- Tool, controls the netfilter framework
- Uses Stateful Packet Inspection (SPI)
- Order matters (top to bottom)

**Syntax**
```
sudo iptables -A <category> -p <protocol> -j <action>
```
- Tables
    - FILTER: standard processing of packets
    - NAT: Rewrite source/destination of packets or track sessions
    - MANGLE: Modify portions of the packet
- Chains (aka categories)
    - INPUT (ingress)
    - OUTPUT (egress)
    - FORWARD (traffic forwarded from one interface to another)
- Table + Chain Combinations
    - FILTER: Input, Output, Forward
    - NAT: Prerouting, Postrouting, Output
    - MANGLE: prerouting, postrouting, input, output, forward
- Timings: stages of processing a packet
    - Prerouting, Input, Output, Forward, Postrouting
- Actions (aka targets)
    - ACCEPT: let packet through
    - REJECT: deny packet, returns packet
    - DROP: "drop it on the floor", never sends back any info packet is logged and deleted
    - LOG: log to syslog
- Filters
    - Source/Destination
        - IP address(es)/subnets
    - Protocol
- Persistence
    - `iptables-save`
    - `iptables-restore`
    - Rules only apply to current running suession
    - Saving:
        - `sudo iptables-save > /etc/sysconfig/iptables` <- Red Hat
        - `sudo iptables-save > /etc/iptables/rules.v4` <- Debian
    - Restore rules:
        - `sudo iptables-restore </etc/sysconfig/iptables` <- Red Hat
        - `sudo iptables-restore </etc/iptables/rules.v4` <- Debian

# Linux Firewall Setup
- Find out the services and protocols that will be used on the server
- `sudo apt-get install iptables-persistent`
    - Will automatically reload iptables upon firewall service restart
    - Can save with `sudo iptables-save`

### iptables examples:
- Add Rule: `sudo iptables -A INPUT -p icmp -j DROP`
    - Example allowing ssh: `sudo iptables -A INPUT -p tcp --dport 22 -j ACCEPT`
    - Example allowing by subnet: `sudo iptables -A INPUT -p icmp --source 10.0.1/24 -j ACCEPT`
- Delete Rule: `sudo iptables -D INPUT -p icmp -j ACCEPT`
    - Delete Rules by LineNumber: 
        - Check linenumbers by using the --line-numbers -L
        - `sudo iptables -D INPUT 2`
- Check Current Rules: `sudo iptables -L`
- Flush All Rules: `sudo iptables -F`
    - Choose a chain to flush by using `sudo iptables -F INPUT`
- Create implicit deny on a chain: `sudo iptables -P INPUT DROP`
- Logging:
    - Setup Logging (order matters):
    ```
    sudo iptables -A INPUT -p -tcp --dport 80 -j LOG --log-level 7 --log-prefix " ***Accept HTTP*** "
    sudo iptables -A INPUT -d hostname -p tcp --dport 80 -j ACCEPT
    ```
    - Check logs: `tail -F /var/log/iptables.log`
    
### ufw uncomplicated firewall (aka Firewall for Dummies)
- Check service: `sudo ufw status`
- Enable ufw: `sudo ufw enable`
    - Service is now enabled
    - You can check iptables with `sudo iptables -L`
        - Now has many rules populated
- Enable log files: `sudo ufw logging HIGH`
    Log level (equal to debug level): `sudo ufw logging HIGH`
- How to check logs: `tail -F /var/log/ufw.log`
- Install log rotate to configure how much space/time logging should take up

**Syntax**
- Allowing new rule: 
    - `sudo ufw allow http`
    - Check rule: `sudo ufw status`
    - Allowing ingress/egress by protocol on a port:
        - `sudo ufw allow proto tcp to any port 22`
        - `sudo ufw allow 1725/udp`
    - Allowing by application
        - `sudo ufw allow from x.x.x.x to any app OpenSSH`